In [137]:
import pandas as pd, numpy as np
import math

def get_days(d0, d1):
    """
    d0 : earlier data
    d1 : later date
    returns : number of days between those two dates.   
    """
    d0 = pd.to_datetime(d0)
    d1 = pd.to_datetime(d1)
    delta = d1 - d0
    return delta.days

# def get_quarter():


train_df = pd.read_csv('./dataset/train.csv')
test_df = pd.read_csv('./dataset/test.csv')

print(f'Train Data: {train_df.shape}')
print(f'Test Data: {test_df.shape}')


train_df['instock_date'] = pd.to_datetime(train_df['instock_date'])
test_df['instock_date'] = pd.to_datetime(test_df['instock_date'])


train_df.dropna(inplace = True)
# test_df.dropna(inplace = True)

old_column_names = train_df.columns.values

name_replacements = {}
for old in old_column_names:
    name_replacements[old] = old.lower()

# column name_replacements
train_df.rename(columns=name_replacements, inplace = True)
test_df.rename(columns=name_replacements, inplace = True)

Train Data: (6368, 15)
Test Data: (3430, 14)


In [138]:
train_df

,product_id,stall_no,instock_date,market_category,customer_name,loyalty_customer,product_category,grade,demand,discount_avail,charges_1,charges_2 (%),minimum_price,maximum_price,selling_price
0,BRAE2NF6JA5GUEXG,37.0,2015-08-22 18:36:12,2,Lillyann,Yes,Fashion,1,68,0.0,376.0,11.0,2983.0,4713.0,4185.947700
1,TUNE8SFB6RJN2HSD,38.0,2016-03-27 21:19:13,24,Klynn,Yes,Fashion,0,51,0.0,397.0,12.0,7495.0,10352.0,9271.490256
2,BRAEAR7WZPQGPBZU,9.0,2015-08-18 19:25:22,447,Ridge,Yes,Child_care,0,10,0.0,250.0,9.0,5752.0,7309.0,6785.701362
3,WATDZ2ZQ8JPDHCTJ,50.0,2016-03-28 21:53:01,23,Abran,Yes,Educational,2,48,0.0,144.0,13.0,5090.0,20814.0,13028.917824
4,JWSEBUKYQPMBZ3RK,7.0,2016-03-29 22:58:53,63,Dustyn,Yes,Repair,1,35,1.0,211.0,4.0,2430.0,9261.0,906.553935
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6363,WATE7VE2D62VEKW8,17.0,2016-03-28 19:55:43,24,Marquell,Yes,Technology,0,47,0.0,614.0,10.0,8174.0,11266.0,10277.520192
6364,SHOEF9G8Z4ZXF6WE,21.0,2015-02-11 15:09:26,55,Theodore,Yes,Child_care,0,35,0.0,269.0,5.0,3564.0,4920.0,4501.837200
6365,PBXEB9ZQ7HHCFXYG,36.0,2015-11-19 17:29:21,358,Galen,No,Repair,3,29,0.0,283.0,3.0,5303.0,14614.0,10218.878775
6366,NKCEAGXHRYTNTMG7,27.0,2015-12-08 04:19:33,452,Posey,Yes,Repair,0,8,0.0,267.0,3.0,4334.0,5849.0,5359.493997


In [139]:
test_df.stall_no.fillna(0, inplace = True)
test_df.charges_1.fillna(0, inplace = True)
test_df.minimum_price.fillna(-1, inplace = True)
test_df['charges_2 (%)'].fillna(0, inplace = True)

[rows, cols] = test_df.shape

for row_idx in range(rows):
    if (test_df.iloc[row_idx, 12] == -1):
        print(test_df.iloc[row_idx, 12:14])
        test_df.iloc[row_idx, 12] = test_df.iloc[row_idx, 13]


test_df.isnull().sum()

minimum_price      -1
maximum_price    8765
Name: 691, dtype: object
minimum_price      -1
maximum_price    5751
Name: 741, dtype: object
minimum_price      -1
maximum_price    4827
Name: 803, dtype: object
minimum_price       -1
maximum_price    14875
Name: 1136, dtype: object
minimum_price      -1
maximum_price    6829
Name: 1476, dtype: object
minimum_price      -1
maximum_price    6178
Name: 1825, dtype: object
minimum_price      -1
maximum_price    7420
Name: 2193, dtype: object
minimum_price      -1
maximum_price    5811
Name: 2320, dtype: object
minimum_price      -1
maximum_price    4891
Name: 2426, dtype: object
minimum_price       -1
maximum_price    10555
Name: 2458, dtype: object
minimum_price      -1
maximum_price    6067
Name: 2720, dtype: object
minimum_price      -1
maximum_price    5559
Name: 2957, dtype: object
minimum_price      -1
maximum_price    2013
Name: 2988, dtype: object
minimum_price      -1
maximum_price    3782
Name: 3138, dtype: object


product_id           0
stall_no             0
instock_date         0
market_category      0
customer_name       53
loyalty_customer     0
product_category     0
grade                0
demand               0
discount_avail       0
charges_1            0
charges_2 (%)        0
minimum_price        0
maximum_price        0
dtype: int64

In [140]:
product_categories = sorted(train_df['product_category'].value_counts().index)
dataset = [train_df, test_df]

In [141]:
for data in dataset:
    data['days_passed'] = [get_days(instock_date, '2016-04-01') for instock_date in data['instock_date']]
    data['quarter'] = [math.ceil(instock_date.month/3.0) for instock_date in data['instock_date']]
    data['product_category'] = [ product_categories.index(category) for category in data['product_category']]
    data['loyalty_customer'] = [1 if loyality=='Yes' else 0 for loyality in data['loyalty_customer']]
#     data['charge_2'] = 

In [143]:
train_df.head(20)

,product_id,stall_no,instock_date,market_category,customer_name,loyalty_customer,product_category,grade,demand,discount_avail,charges_1,charges_2 (%),minimum_price,maximum_price,selling_price,days_passed,quarter
0,BRAE2NF6JA5GUEXG,37.0,2015-08-22 18:36:12,2,Lillyann,1,3,1,68,0.0,376.0,11.0,2983.0,4713.0,4185.947700,222,3
1,TUNE8SFB6RJN2HSD,38.0,2016-03-27 21:19:13,24,Klynn,1,3,0,51,0.0,397.0,12.0,7495.0,10352.0,9271.490256,4,1
2,BRAEAR7WZPQGPBZU,9.0,2015-08-18 19:25:22,447,Ridge,1,0,0,10,0.0,250.0,9.0,5752.0,7309.0,6785.701362,226,3
3,WATDZ2ZQ8JPDHCTJ,50.0,2016-03-28 21:53:01,23,Abran,1,2,2,48,0.0,144.0,13.0,5090.0,20814.0,13028.917824,3,1
4,JWSEBUKYQPMBZ3RK,7.0,2016-03-29 22:58:53,63,Dustyn,1,8,1,35,1.0,211.0,4.0,2430.0,9261.0,906.553935,2,1
5,SHIECAYW3VZWGZGX,23.0,2015-09-25 16:31:46,162,Brilee,0,9,1,49,0.0,633.0,6.0,3959.0,6104.0,5631.247872,188,3
6,SHOE4FZGGTE7HJXN,33.0,2014-12-08 12:25:52,462,Zoe,0,1,3,57,0.0,263.0,13.0,2006.0,5257.0,3874.264992,479,4
7,SHOE6QGAYFFMBWG6,4.0,2015-09-22 14:29:06,23,Koby,1,2,1,67,0.0,104.0,15.0,3596.0,5921.0,4820.943600,191,3
8,KRTEBSMKCGFESHYX,26.0,2016-03-28 21:48:01,156,Tiffani,1,0,0,53,0.0,310.0,6.0,6965.0,10194.0,8837.724240,3,1
9,BRAE3P8Y8UVRY43K,5.0,2015-08-19 22:16:52,275,Nigel,1,0,0,84,1.0,321.0,8.0,1103.0,5430.0,530.761862,225,3


In [158]:
features = ['stall_no','market_category', 'loyalty_customer', 'product_category', 'grade',
            'demand', 'discount_avail', 'charges_1', 'charges_2 (%)',
            'minimum_price', 'maximum_price', 'days_passed', 'quarter']

train_df_norm = (train_df[features] - train_df[features].min())/(train_df[features].max()-train_df[features].min())
test_df_norm = (test_df[features] - test_df[features].min())/(test_df[features].max()-test_df[features].min())


X = train_df_norm[features].copy()
y_min = train_df['selling_price'].min()
y_max = train_df['selling_price'].max()
y_range = (y_max-y_min)

y = (train_df['selling_price'] -y_min)/(y_max-y_min)

In [157]:
train_df_norm

,stall_no,market_category,loyalty_customer,product_category,grade,demand,discount_avail,charges_1,charges_2 (%),minimum_price,maximum_price,days_passed,quarter
0,0.734694,0.004246,1.0,0.333333,0.333333,0.676768,0.0,0.502732,0.6250,0.163783,0.073774,0.287760,0.666667
1,0.755102,0.050955,1.0,0.333333,0.000000,0.505051,0.0,0.540984,0.6875,0.422262,0.170319,0.003906,0.000000
2,0.163265,0.949045,1.0,0.000000,0.000000,0.090909,0.0,0.273224,0.5000,0.322411,0.118220,0.292969,0.666667
3,1.000000,0.048832,1.0,0.222222,0.666667,0.474747,0.0,0.080146,0.7500,0.284487,0.349438,0.002604,0.000000
4,0.122449,0.133758,1.0,0.888889,0.333333,0.343434,1.0,0.202186,0.1875,0.132104,0.151640,0.001302,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6363,0.326531,0.050955,1.0,1.000000,0.000000,0.464646,0.0,0.936248,0.5625,0.461159,0.185968,0.002604,0.000000
6364,0.408163,0.116773,1.0,0.000000,0.000000,0.343434,0.0,0.307832,0.2500,0.197067,0.077318,0.537760,0.000000
6365,0.714286,0.760085,0.0,0.888889,1.000000,0.282828,0.0,0.333333,0.1250,0.296689,0.243289,0.171875,1.000000
6366,0.530612,0.959660,1.0,0.888889,0.000000,0.070707,0.0,0.304189,0.1250,0.241178,0.093224,0.147135,1.000000


## Linear Regressor

In [166]:
from sklearn.linear_model import LinearRegression


reg = LinearRegression()
reg.fit(X, y)
reg.score(X, y)
reg.coef_
reg.intercept_

0.24916242489570087

In [168]:
reg.coef_

X_test = test_df_norm[features]
y_predict_norm = reg.predict(X_test)

y_predict = y_min + y_predict_norm*y_range
y_predict

array([ 5251.27897092,  3261.98118158, 18407.80640797, ...,
       10871.98515618,  6128.47755969,  5591.57031051])

In [173]:
y_predict = abs(y_predict)
y_predict[y_predict <0] 

sub = pd.DataFrame({'Product_id': test_df.product_id,'Selling_Price': y_predict})
sub.to_csv('submission.csv', index = False)

array([], dtype=float64)

## SGD Regressor

In [178]:
import numpy as np
from sklearn.linear_model import SGDRegressor


# Always scale the input. The most convenient way is to use a pipeline.
sgd_regressor = SGDRegressor(max_iter=1000, tol=1e-3)
sgd_regressor.fit(X, y)
sgd_regressor.get_params()

{'alpha': 0.0001,
 'average': False,
 'early_stopping': False,
 'epsilon': 0.1,
 'eta0': 0.01,
 'fit_intercept': True,
 'l1_ratio': 0.15,
 'learning_rate': 'invscaling',
 'loss': 'squared_loss',
 'max_iter': 1000,
 'n_iter_no_change': 5,
 'penalty': 'l2',
 'power_t': 0.25,
 'random_state': None,
 'shuffle': True,
 'tol': 0.001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [182]:
y_predict2 = y_min + sgd_regressor.predict(X_test)*y_range
y_predict2 = abs(y_predict2)
sub = pd.DataFrame({'Product_id': test_df.product_id,'Selling_Price': y_predict2})
sub.to_csv('submission_sgd_regressor.csv', index = False)

# Evaluation Metrics: RMSLE 
[RMSLE Explained](https://medium.com/analytics-vidhya/root-mean-square-log-error-rmse-vs-rmlse-935c6cc1802a)
![](https://miro.medium.com/max/980/0*AUzyQ1rc6mpQVYfn)